In [1]:
library(caret)

Loading required package: ggplot2


In [2]:
data <- read.csv('Data/MNIST/train.csv')
head(data)

label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# Partition the dataset 75%/25% split in order to create a train and test dataset
partition <- createDataPartition(y = data$label, p = .75,  list = FALSE)

# Get the train and test dataset
train <- data[partition, ] # This gives us the 75%
test <- data[-partition, ] # This gives us the remaining 25%

In [4]:
head(train)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
mnistModelLR <- rxLogisticRegression(
    formula = label ~ .,
    data = train, 
    type = "multiClass",)

Automatically adding a MinMax normalization transform, use 'norm=Warn' or 'norm=No' to turn this behavior off.
Beginning processing data.
Rows Read: 31502, Read Time: 0.004, Transform Time: 0
Beginning processing data.
Automatically converting column 'label' into a factor.
Beginning processing data.
Rows Read: 31502, Read Time: 0.003, Transform Time: 0
Beginning processing data.
Beginning processing data.
Rows Read: 31502, Read Time: 0.004, Transform Time: 0
Beginning processing data.
Beginning processing data.
Rows Read: 31502, Read Time: 0.003, Transform Time: 0
Beginning processing data.
LBFGS multi-threading will attempt to load dataset into memory. In case of out-of-memory issues, turn off multi-threading by setting trainThreads to 1.
Beginning optimization
num vars: 7850
improvement criterion: Mean Improvement
L1 regularization selected 3256 of 7850 weights.
Not training a calibrator because it is not needed.
Elapsed time: 00:00:13.0507502
Elapsed time: 00:00:00.2960999


In [6]:
mnistModelNN <- rxNeuralNet(
    formula = label ~ .,
    data = train, 
    type = "multiClass",)

Automatically adding a MinMax normalization transform, use 'norm=Warn' or 'norm=No' to turn this behavior off.
Beginning processing data.
Rows Read: 31502, Read Time: 0.003, Transform Time: 0
Beginning processing data.
Automatically converting column 'label' into a factor.
Beginning processing data.
Rows Read: 31502, Read Time: 0.004, Transform Time: 0
Beginning processing data.
Beginning processing data.
Rows Read: 31502, Read Time: 0.004, Transform Time: 0
Beginning processing data.
Beginning processing data.
Rows Read: 31502, Read Time: 0.003, Transform Time: 0
Beginning processing data.
Using: SSE Math

***** Net definition *****
  input Data [784];
  hidden H [100] sigmoid { // Depth 1
    from Data all;
  }
  output Result [10] softmax { // Depth 0
    from H all;
  }
***** End net definition *****
Input count: 784
Output count: 10
Output Function: SoftMax
Loss Function: LogLoss
PreTrainer: NoPreTrainer
___________________________________________________________________
Starting 

In [7]:
#mnistModelEN <- rxEnsemble(
#    formula = label ~ .,
#    data = train, 
#    type = "multiClass", 
#    trainers = list(fastForest(), fastForest(), logisticRegression()))

In [9]:
scoreLR <- rxPredict(mnistModelLR, data = test, extraVarsToWrite = "label")
scoreNN <- rxPredict(mnistModelNN, data = test, extraVarsToWrite = "label")
#scoreEN <- rxPredict(mnistModel, data = test, extraVarsToWrite = "label")

Beginning processing data.
Rows Read: 10498, Read Time: 0.013, Transform Time: 0
Beginning processing data.
Elapsed time: 00:00:01.0645476
Finished writing 10498 rows.
Writing completed.
Beginning processing data.
Rows Read: 10498, Read Time: 0.013, Transform Time: 0
Beginning processing data.
Elapsed time: 00:00:01.1413963
Finished writing 10498 rows.
Writing completed.


In [10]:
print(sum(scoreLR$label == scoreLR$PredictedLabel)/dim(scoreLR)[1])
print(sum(scoreNN$label == scoreNN$PredictedLabel)/dim(scoreNN)[1])
#print(sum(scoreEN$label == scoreEN$PredictedLabel)/dim(scoreEN)[1])

[1] 0.9182701
[1] 0.9653267


In [13]:
classMnist<-function(input){
    score <- rxPredict(mnistModelNN, data = input, extraVarsToWrite = 'label')
    return(score$PredictedLabel)
}

In [16]:
remoteLogin("http://localhost:12800", 
            username = "admin", 
            password = "Pass@word1",
            session = FALSE)

api <- updateService(
     "classMnist",
     code = classMnist,
     model = mnistModelNN,
     inputs = list(input = "data.frame"),
     outputs = list(PredictedLabel = "numeric"),
     v = "v1.0.0"
)